In [ ]:
# my task is to fill in this py. 
# worker is a part of chatbot app that processes user msgs and documents 
# worker uses langchain library to handle pdf and llms 

# placeholder for Watsonx_API and Project_id incase you need to use the code outside this environment
Watsonx_API = ""
Project_id= "skills-network"

# initialization 
# Function to initialize the language model and its embeddings
def init_llm():
    global llm_hub, embeddings
    my_credentials = {
        "url"    : "https://us-south.ml.cloud.ibm.com"
    } # 질문 : my credentials을 하는 이유가 뭐야? 
    params = {
            GenParams.MAX_NEW_TOKENS: 500, # The maximum number of tokens that the model can generate in a single run.
            GenParams.TEMPERATURE: 0.1,   # A parameter that controls the randomness of the token generation. A lower value makes the generation more deterministic, while a higher value introduces more randomness.
        }
    LLAMA2_model = Model(
            model_id= 'meta-llama/llama-3-8b-instruct', 
            credentials=my_credentials,
            params=params,
            project_id="skills-network",  # <--- NOTE: specify "skills-network" as your project_id
            )
    llm_hub = WatsonxLLM(model=LLAMA2_model)

#Initialize embeddings
embeddings = HuggingFaceInstructEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": DEVICE}
    )
# 질문 : kwargs은 무슨뜻이야? 

# Document processing 
def process_document(document_path):
    global conversation_retrieval_chain 

    # document loading 
    loader = PyPDFLoader(document_path)

    documents = loader.load()

    # document splitting 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    texts = text_splitter.split_documents(documents)

    # vector store creation 
    db = Chroma.from_documents(texts, embedding=embeddings) 

    # QA chain that uses LLM and retriever for QA # 질문 : 이 코드에서 정확히 QAchain의 용도를 모르겠어. 
    conversation_retrieval_chain = RetrievalQA.from_chain_type(
        llm=llm_hub,
        chain_type="stuff",
        retriever= db.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25}),
        # 질문 : retriever code를 이해하기 어려워 쉽게 설명해줘. 
        return_source_documents=False
    )


# user prompt processing 
def process_prompt(prompt):
    global conversation_retrieval_chain
    global chat_history 

    # Pass the prompt and the chat history to the conversation_retrieval_chain object
    output = conversation_retrieval_chain({"question": prompt, "chat_history": chat_history})
    answer =  output["result"]

    # update the chat history 
    chat_history.append((prompt, answer)) # 질문 : udpate하는 용도를 알려줘 . 

    # return model's answer 
    return answer